In [1]:
import sys
import os
sys.path.append(r"..")

from utils import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score

path_build = "../../data/tencent2020/build/"
path_save = "../../data/tencent2020/save/"
pickle_path = "../../data/tencent2020/pickle"
sub_path = "../../data/tencent2020/sub"

train_preliminary_p = path_build + "train_preliminary/"

if not os.path.exists(pickle_path):
    os.mkdir(pickle_path) 
if not os.path.exists(sub_path):
    os.mkdir(sub_path)


In [2]:
user_creative_ids = pd.read_pickle(f"{pickle_path}/user_creative_ids.pickle")
user = pd.read_csv(train_preliminary_p + "user.csv", encoding='utf-8')
user_creative_ids = user_creative_ids.merge(user, how='left', on='user_id')
user_creative_ids.head()

,user_id,creative_id,age,gender
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169...",4.0,1.0
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ...",10.0,1.0
2,3,"[661347, 808612, 710859, 825434, 593522, 72694...",7.0,2.0
3,4,"[39588, 589886, 574787, 1892854, 1962706, 2264...",5.0,1.0
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...",4.0,1.0


In [25]:
cv_gender = CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b",min_df=25,ngram_range=(1,1))
tf_matrix_gender = cv_gender.fit_transform(user_creative_ids['creative_id'].map(lambda x:' '.join([str(i) for i in x])).values)
print(tf_matrix_gender.shape)
data_tfidf_gender = TfidfTransformer(use_idf=True).fit_transform(tf_matrix_gender)

cv_age = CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b",min_df=100,ngram_range=(1,1))
tf_matrix_age = cv_age.fit_transform(user_creative_ids['creative_id'].map(lambda x:' '.join([str(i) for i in x])).values)
print(tf_matrix_age.shape)
data_tfidf_age = TfidfTransformer(use_idf=True).fit_transform(tf_matrix_age)


(1900000, 262005)
(1900000, 71208)


In [20]:
## 奇异值分解 降维
#(U, S, VT) = np.linalg.svd(data_tfidf)
#U.shape, S.shape, VT.shape
from scipy.sparse import coo_matrix
aa = data_tfidf.tocoo().toarray()
aa.shape
#data_tfidf.toarray().shape

MemoryError: 

训练

In [30]:
offline = False
if offline:
    train_filter = 700000
else:
    train_filter = 900000
gender_clf = MultinomialNB().fit(data_tfidf_gender[0:train_filter], user_creative_ids['gender'][0:train_filter])
age_clf = MultinomialNB().fit(data_tfidf_age[0:train_filter], user_creative_ids['age'][0:train_filter])

测试

In [31]:
if offline:
    train_filter = 700000
    gender_p = gender_clf.predict(data_tfidf_gender[train_filter:900000])
    age_p = age_clf.predict(data_tfidf_age[train_filter:900000])
else:
    train_filter = 900000
    gender_p = gender_clf.predict(data_tfidf_gender[train_filter:])
    age_p = age_clf.predict(data_tfidf_age[train_filter:])

评估提交

In [34]:
if offline:
    gender_accuracy = accuracy_score(user_creative_ids['gender'][train_filter:900000], gender_p)
    age_accuracy = accuracy_score(user_creative_ids['age'][train_filter:900000], age_p)
    print(gender_accuracy, age_accuracy, gender_accuracy+age_accuracy)
else:
    train_gender_p = gender_clf.predict(data_tfidf_gender[:train_filter])
    train_age_p = age_clf.predict(data_tfidf_age[:train_filter])
    
    gender_accuracy = accuracy_score(user_creative_ids['gender'][:train_filter], train_gender_p)
    age_accuracy = accuracy_score(user_creative_ids['age'][:train_filter], train_age_p)
    print(gender_accuracy, age_accuracy, gender_accuracy+age_accuracy)
    sub = pd.DataFrame()
    sub['user_id'] = range(3000001,4000001)
    sub['predicted_age'] = age_p
    sub['predicted_gender'] = gender_p
    sub.to_csv(f"{sub_path}/submission.csv", index=False, encoding='utf-8')
    print(sub.shape)
    sub.head()

0.9203522222222222 0.4466233333333333 1.3669755555555556
(1000000, 3)


0.861665 0.280975 1.14264  
0.90131 0.31792 1.21923  min_df=5  
0.909795 0.35963 1.269425  min_df=25    (1900000, 262005)  
0.908905 0.367675 1.27658  min_df=50  (1900000, 138555)  
0.905905 0.36898 1.2748849999999998  100 (1900000, 71208)  
0.909795 0.36898 1.278775  线上1.285234  25和100的结合  



## lgb 训练

In [36]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'n_estimators': 5000,
    'metric': 'mae',
    'learning_rate': 0.01,
    'min_child_samples': 5,
    'min_child_weight': 0.01,
    'subsample_freq': 1,
    'num_leaves': 31,
    'max_depth': -1,
    'subsample': 0.8,
    'colsample_bytree': 0.6,
    'reg_alpha': 0,
    'reg_lambda': 5,
    'verbose': -1,
    'random_state': 4590,
    'n_jobs': 4,
}

In [74]:
user_creative_ids['gender'][0:train_filter].unique()

array([1., 2.])

In [39]:
test_x =  data_tfidf_age[train_filter:900000]
test_x.shape
train_filter

900000

In [40]:
cv = True
model_type = 'lgb'
seed = 2020
train_filter = 700000
train_x = data_tfidf_age[0:train_filter]
train_y = user_creative_ids['gender'][0:train_filter]
test_x =  data_tfidf_age[train_filter:900000]
if cv:
    n_fold = 5
    count_fold = 0
    preds_list = list()
    oof = np.zeros(train_x.shape[0])
    kfolder = KFold(n_splits=n_fold, shuffle=True, random_state=seed)
    kfold = kfolder.split(train_x, train_y)
    for train_index, vali_index in kfold:
        print("training......fold",count_fold)
        count_fold = count_fold + 1
        k_x_train = train_x[train_index]
        k_y_train = train_y[train_index]
        k_x_vali = train_x[vali_index]
        k_y_vali = train_y[vali_index]
        if model_type == 'lgb':
            lgb_model = lgb.LGBMRegressor(**lgb_params)
            lgb_model = lgb_model.fit(k_x_train, k_y_train, eval_set=[(k_x_vali, k_y_vali)],
                                      early_stopping_rounds=200, verbose=False, eval_metric="mae",
                                      )
            k_pred = lgb_model.predict(k_x_vali, num_iteration=lgb_model.best_iteration_)
            pred = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration_)
        preds_list.append(pred)
        oof[vali_index] = k_pred

training......fold 0


KeyboardInterrupt: 

In [84]:
preds_columns = ['preds_{id}'.format(id=i) for i in range(n_fold)]
preds_df = pd.DataFrame(data=preds_list)
preds_df = preds_df.T
preds_df.columns = preds_columns
preds = list(preds_df.mean(axis=1))


In [91]:
preds22 = np.rint(preds)
gender_accuracy = accuracy_score(user_creative_ids['gender'][train_filter:900000], preds22)
gender_accuracy,preds22.shape

(0.781365, (200000,))